# Discounting Trends

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
# %load_ext autoreload
# the "1" means: always reload modules marked with "%aimport"
# %autoreload 1

#from __future__ import absolute_import, division, print_function
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import GridSpec
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys
import warnings
from sqlalchemy import create_engine
#from pivottablejs import pivot_ui


In [2]:
sns.set();
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_context("poster", font_scale=1.3)
sns.set_style("whitegrid")
warnings.filterwarnings('ignore')

## Load Data

In [3]:
from decouple import config

In [4]:
#val = !cat '/home/jovyan/.env'
#db=val[0].split('=')[1];
db=config('DATABASE_URL')

In [5]:
engine = create_engine(db)

In [6]:
sql = '''
SELECT  t.Item, t.PriceMethod, sum(NetSalesAmt) as ext_price, sum(ShippedQty) as ext_qty, count(*) freq
FROM BRS_TransactionDW t 
inner join BRS_Item i
on i.Item = t.Item
where 
CalMonth between 201901 and 201912 AND
FreeGoodsInvoicedInd = 0 and
DocType like 'S%' AND
i.SalesCategory = 'MERCH' AND
1=1
group by t.item, t.PriceMethod, NetSalesAmt / ShippedQty
;

'''

In [7]:
%time df = pd.read_sql_query(sql, engine);

CPU times: user 3.34 s, sys: 97.3 ms, total: 3.44 s
Wall time: 5.43 s


In [8]:
print (df.dtypes)
df.describe()


Item            object
PriceMethod     object
ext_price      float64
ext_qty          int64
freq             int64
dtype: object


ext_price        ext_qty           freq
count  480536.000000  480536.000000  480536.000000
mean      516.759956      12.417786       3.859199
std      2897.202982     110.894552      21.462994
min         0.000000       1.000000       1.000000
25%        58.990000       1.000000       1.000000
50%       141.700000       2.000000       1.000000
75%       359.080000       6.000000       3.000000
max    503842.800000   29904.000000    7956.000000

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480536 entries, 0 to 480535
Data columns (total 5 columns):
Item           480536 non-null object
PriceMethod    480536 non-null object
ext_price      480536 non-null float64
ext_qty        480536 non-null int64
freq           480536 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 18.3+ MB


## Trends

In [11]:
df.PriceMethod.unique()

array(['8', 'C', 'Q', 'R', 'P', '*', 'S', '$', 'D'], dtype=object)

In [ ]:
df.groupby('PriceMethod')

In [18]:
sns.barplot(data=df,x='ext_price',y='PriceMethod',orient='h',)

SyntaxError: unexpected EOF while parsing (<ipython-input-18-0be5ff1a3d38>, line 1)

In [15]:
%time df2 = pd.read_sql_query('select * from BRS_Item', engine);

CPU times: user 8.56 s, sys: 333 ms, total: 8.89 s
Wall time: 8.9 s


In [35]:
df2.shape

(226680, 40)